# 1. Introduction
## 1.1 Background
New york is one of the world's biggest city with millisons of people live in it. People need different business of venues in the city. For a venue, you have to pick the right place to open the business in order to be successful. It is valuable to predict where is appropiate to open a specific business.
## 1.2 Problem

Data that might contribute to determining where to open a business might include the location, existing other venues, the number of other venues, and the rating of them. This project aims to predict where a neighorhood is good for a certain venue like restaurant.
## 1.3 Interest
It is clear that those who want to open any kind of venue would be interested in where is good to open their business.

# 2. Data acquisition
## 2.1 Data source

We get the neighborhood information from https://cocl.us/new_york_dataset. However, we don't have the business information from the dataset. So we need to get venue information from foursquare based on the center location of the neighborhood.

## 2.2 Data cleaning

Data downloaded or scraped from both the webpage.

First, we don't have the location information from the data, so we need to look up the latitude and longitude from google programatically to fill in the table.

Second, we use the foursquare to look up top 100 popular venues in one neighborhood and then one-hot-encode them. 


## 2.3 Feature selection

There are a few issues with the features. 


There is still some categories that is so small in all neighborhood. Which I believe they are outliers, I'll just remove them.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

ModuleNotFoundError: No module named 'folium'

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
CLIENT_ID = 'KARX01P1ZZKKWCBEI0ANBTSS14RMKO5FOG3H213ANVLLCN5L' # your Foursquare ID
CLIENT_SECRET = 'MU3BBAU1JAUCH0P1NSSAYHZOKG3SXM1NRP3WG5DOS4O1ZE1T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here

neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                       latitudes=neighborhoods['Latitude'],
                                       longitudes=neighborhoods['Longitude']
                                  )

neighborhoods_venues.head()

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [7]:
neighborhoods_venues.to_csv('n.csv')

In [33]:
import pandas as pd
neighborhoods_venues = pd.read_csv('n.csv')
# one hot encoding
onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
# get a list of columns
cols = list(onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))
onehot = onehot[cols]

onehot = onehot.groupby("Neighborhood").mean()
onehot.shape

(302, 425)

In [175]:
import numpy as np
sum_ = onehot.sum().sort_values().to_frame()
p = np.percentile(sum_, [10, 99])
sum_ = sum_[sum_[0]>p[0]]
onehot = onehot[sum_.index]
onehot.head()

,Flea Market,Newsstand,Club House,Street Art,Piano Bar,High School,Udon Restaurant,Coworking Space,Piercing Parlor,Austrian Restaurant,Gaming Cafe,Herbs & Spices Store,Gluten-free Restaurant,Insurance Office,Souvlaki Shop,Tex-Mex Restaurant,Multiplex,Resort,Romanian Restaurant,Varenyky restaurant,Lebanese Restaurant,Egyptian Restaurant,Ski Area,Racetrack,Church,Indian Chinese Restaurant,Shop & Service,Arts & Entertainment,Laundry Service,Design Studio,Adult Boutique,Strip Club,Moroccan Restaurant,Shabu-Shabu Restaurant,Cha Chaan Teng,Camera Store,Kosher Restaurant,Sri Lankan Restaurant,Tibetan Restaurant,Public Art,Himalayan Restaurant,Caucasian Restaurant,Factory,Pakistani Restaurant,Gay Bar,Tourist Information Center,Pool Hall,Bridge,Tanning Salon,Australian Restaurant,Chocolate Shop,Pet Café,Outdoor Sculpture,Bike Trail,Kebab Restaurant,Library,Memorial Site,Israeli Restaurant,Non-Profit,Doctor's Office,Opera House,Waterfront,Tailor Shop,Used Bookstore,Japanese Curry Restaurant,Halal Restaurant,Leather Goods Store,College Basketball Court,Music School,Vape Store,Recording Studio,Office,Indonesian Restaurant,Social Club,Tiki Bar,Szechuan Restaurant,Polish Restaurant,Warehouse Store,Pie Shop,Colombian Restaurant,Fountain,College Academic Building,Hawaiian Restaurant,Hot Dog Joint,Heliport,Climbing Gym,Malay Restaurant,Automotive Shop,Motel,Track,Garden Center,Rental Service,Whisky Bar,Cantonese Restaurant,Bed & Breakfast,Hotel Pool,Train,College Cafeteria,Pedestrian Plaza,Mini Golf,Print Shop,Waste Facility,Taiwanese Restaurant,Paella Restaurant,Jewish Restaurant,Exhibit,Massage Studio,Other Great Outdoors,Sake Bar,Food Stand,Campground,Brazilian Restaurant,Bike Shop,Beer Store,Distillery,Gymnastics Gym,Ethiopian Restaurant,Boxing Gym,Pet Service,Empanada Restaurant,Buffet,Organic Grocery,Motorcycle Shop,Shanghai Restaurant,Airport Terminal,Cafeteria,Bike Rental / Bike Share,Poke Place,Hostel,Theme Park Ride / Attraction,Farm,Platform,Lake,Skate Park,Weight Loss Center,General Entertainment,Bistro,Field,African Restaurant,Burrito Place,Argentinian Restaurant,Rock Club,Hockey Field,Creperie,Arepa Restaurant,Bridal Shop,Hotpot Restaurant,Sports Club,Gym Pool,Afghan Restaurant,Tea Room,Comedy Club,Butcher,Toll Plaza,Nightlife Spot,Dumpling Restaurant,Baseball Stadium,Building,Dim Sum Restaurant,Cheese Shop,Beer Garden,School,Indie Movie Theater,Tennis Stadium,Snack Place,Residential Building (Apartment / Condo),German Restaurant,Roof Deck,Speakeasy,Jazz Club,Concert Hall,Music Store,Comfort Food Restaurant,Cajun / Creole Restaurant,Indie Theater,Big Box Store,Filipino Restaurant,Cupcake Shop,Outdoors & Recreation,Dosa Place,Fruit & Vegetable Store,Soccer Field,Check Cashing Service,Irish Pub,Sculpture Garden,Hobby Shop,Karaoke Bar,Storage Facility,Health Food Store,Russian Restaurant,Garden,Falafel Restaurant,Fish & Chips Shop,Tattoo Parlor,Theme Park,Record Shop,Beer Bar,Lingerie Store,Recreation Center,Toy / Game Store,Other Nightlife,Cuban Restaurant,Turkish Restaurant,Event Service,Cycle Studio,Movie Theater,Eye Doctor,Surf Spot,Nail Salon,Skating Rink,Wings Joint,Pilates Studio,Gastropub,Dry Cleaner,Soup Place,Event Space,Business Service,Tapas Restaurant,Food Court,Arcade,IT Services,Other Repair Shop,Noodle House,Frozen Yogurt Shop,Video Store,Paper / Office Supplies Store,Museum,Art Museum,Rest Area,Miscellaneous Shop,Eastern European Restaurant,Mattress Store,Salad Place,Hookah Bar,Hotel Bar,Lawyer,Hardware Store,Ramen Restaurant,Men's Store,Monument / Landmark,Home Service,Accessories Store,Antique Shop,Fish Market,Laundromat,Kids Store,Jewelry Store,Pier,Southern / Soul Food Restaurant,Golf Course,Taco Place,Farmers Market,Shipping Store,Performing Arts Venue,Flower Shop,Nightclub,Construction & Landscaping,Health & Beauty Service,Sports Bar,History Museum,Electronics Store,Board Shop,Supplement Shop,Theater,Peruvian Restaurant,Music Venue,Smoke Shop,Arts & Crafts Store,Video Game Store,Women's Store,Trail,BBQ Joint,H

# 3. Exploratory Data Analysis
## 3.1 Relationship between popularity and location

In [189]:
# Let us sort by "resturants" and get the top 20
top20 = onehot.sort_values("Restaurant", ascending=False)[:20].index
top20n = neighborhoods[neighborhoods['Neighborhood'].isin(top20)]
top20n

,Borough,Neighborhood,Latitude,Longitude
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
22,Bronx,Port Morris,40.801664,-73.913221
24,Bronx,Hunts Point,40.809730,-73.883315
51,Brooklyn,Brighton Beach,40.576825,-73.965094
60,Brooklyn,Brownsville,40.663950,-73.910235
80,Brooklyn,Borough Park,40.633131,-73.990498
98,Brooklyn,Ocean Parkway,40.613060,-73.968367
134,Queens,Corona,40.742382,-73.856825
145,Queens,Rego Park,40.728974,-73.857827


In [193]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium
map_newyork = folium.Map(location=[40.874294, -73.829939], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(top20n['Latitude'], top20n['Longitude'], top20n['Borough'], top20n['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**In case you don't see the map, it seems they are widely spread in new york city so that it doesn't make a lot sense to use location to predict if a venus would be popular in one neighborhood or not.**

## 3.2 Relationship between popularity and similarity

In [231]:
import scipy.stats as sstats
def pearsonr(l1, l2):
    print(l1)
    Sxx = sum([i**2 for i in l1]) - pow(sum(l1),2)/float(len(l1))
    Syy = sum([i**2 for i in l2]) - pow(sum(l2),2)/float(len(l2))
    Sxy = sum( i*j for i, j in zip(l1, l2)) - sum(l1)*sum(l2)/float(len(l1))
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        return Sxy/sqrt(Sxx*Syy)
    else:
        return 0
# find where the most popular restuarant is
top1 = onehot.sort_values("Restaurant", ascending=False)[:1]
del top1['Restaurant']
sstats.pearsonr(top1.values.tolist()[0], top1.values.tolist()[0])
onehot_nr = onehot.copy()
del onehot_nr['Restaurant']

In [242]:
onehot_nrc = onehot_nr.copy()
onehot_nrc['closeness'] = onehot_nrc.apply(lambda x: sstats.pearsonr(top1.values.tolist()[0], x.tolist())[0], axis=1)
onehot_nrc = onehot_nrc.sort_values('closeness', ascending=False)
onehot_nrc[['closeness']][:40].sort_index()


,closeness
Neighborhood,
Bayswater,0.412369
Brownsville,1.000000
Castle Hill,0.350742
Claremont Village,0.343169
Clason Point,0.402737
Concourse,0.353356
East Flatbush,0.382339
Flatlands,0.333383
Fulton Ferry,0.391748


In [256]:
top40 = onehot.sort_values("Restaurant", ascending=False)[:40].index
np.unique(onehot_nrc[:40].index.isin(top40), return_counts=True)

(array([False,  True]), array([34,  6]))

## 3.3 Relationship between popularity and unsupervised cluster

In [194]:
# set number of clusters
kclusters = 15

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(onehot)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 7, 13, 11, 11,  7,  1,  1,  1,  1, 13], dtype=int32)

In [197]:
onehot_cluster=onehot.copy()
onehot_cluster.insert(0, 'Cluster Label', kmeans.labels_)
top20c = onehot_cluster.sort_values("Restaurant", ascending=False)[:20]
top20c[['Cluster Label']]

,Cluster Label
Neighborhood,
Brownsville,1
Midland Beach,1
Annadale,13
Cambria Heights,8
New Dorp Beach,7
Heartland Village,1
Hunts Point,13
Madison,7
Brighton Beach,1


In [203]:
onehot_cluster['Cluster Label'].value_counts()

1     114
13     98
7      36
8      16
11     15
4       7
0       7
3       2
14      1
12      1
10      1
9       1
6       1
5       1
2       1
Name: Cluster Label, dtype: int64

**Clearly, unsupervised cluster labels capture where are good for resturants. Those with label 1 and 13 are mostly popular for resturants. You should consider those. But most neighborhoods belong to 1 and 13. It is also vagour where to put your restaurant.**

# 4. Predictive Modeling

After exploring the model, it seems the similary way is most promising method to predict if a neighborhood is good for a specific venue.

Let us explore other venues like 'Office', 'Pharmacy'

In [259]:
def predict_venue(v):
    # find where the most popular restuarant is
    top1 = onehot.sort_values(v, ascending=False)[:1]
    del top1[v]
    onehot_nr = onehot.copy()
    del onehot_nr[v]
    onehot_nrc = onehot_nr.copy()
    onehot_nrc['closeness'] = onehot_nrc.apply(lambda x: sstats.pearsonr(top1.values.tolist()[0], x.tolist())[0], axis=1)
    onehot_nrc = onehot_nrc.sort_values('closeness', ascending=False)
    top40 = onehot.sort_values(v, ascending=False)[:40].index
    print("predict rate of ", v, np.unique(onehot_nrc[:40].index.isin(top40), return_counts=True))
predict_venue('Office')
predict_venue('Pharmacy')

predict rate of  Office (array([False,  True]), array([32,  8]))
predict rate of  Pharmacy (array([False,  True]), array([36,  4]))


# 5. Conclusions
In this study, I analyzed the relationship between popularity of certain venue in a neighborhood and their location, unsupervised cluster label, similarity other than the venue. I noticed that none of them can clearly predict if a venue has already been popular in a neighborhood. But similarity model in recommendation system has the best potential. So I build the predictive model based on similarity. The model's predictive ability is aound 10% - 20%. However, it doesn't mean the model is not useful. Maybe it has uncovered potential good places for the future.

# 6. Future Directions
To further understand if the model is useful or not, I need to further collect more data over years so as to understand if a place would become popular so as to either prove the existing model still has potential to uncover potentials people cannot easily understand. Or create regression models using machine learning so as to predict if a venue would become popular in the future based on its current status.

More features might be needed like if certain venues' rating is high or low might also need consideration.